In [1]:
import sympy as sp
from sympy import sin, cos, pi
from sympy.interactive import printing
import pickle
import numpy as np
import scipy as sc
import scipy.interpolate
from scipy.integrate import odeint
import matplotlib.pyplot as pl

import symbtools as st
import symbtools.modeltools as mt
import symbtools.noncommutativetools as nct

In [1]:
t = sp.Symbol('t')
Np = 1
Nq = 1
n = Np + Nq
pp = st.symb_vector("p1:{0}".format(Np+1))
qq = st.symb_vector("q1:{0}".format(Nq+1))
aa = st.symb_vector("a1:{0}".format(Nq+1))
ww = st.symb_vector("w1:{0}".format(Nq+1))

ttheta = st.row_stack(pp, qq) ##:T
tthetad = st.time_deriv(ttheta, ttheta) ##:T
tthetadd = st.time_deriv(ttheta, ttheta, order=2) ##:T
st.make_global(ttheta, tthetad)

NameError: name 'sp' is not defined

In [3]:
params = sp.symbols('m1, m2, s2, g') # m1 wagen, m2 pole
st.make_global(params)

tau1 = sp.Symbol("tau1")

In [4]:
#Einheitsvektoren
ex = sp.Matrix([1,0])
ey = sp.Matrix([0,1])


# Koordinaten der Schwerpunkte und Gelenke
S1 = ex*q1 # Schwerpunkt Wagen
G2 = S1 # Pendel-Gelenk

# Schwerpunkt des Pendels (Pendel zeigt für kleine Winkel nach #! OBEN)
S2 = G2 +sp.Matrix([s2/2 * sin(p1), s2/2*cos(p1)]) #- mt.Rz(p1)*(ey)*s2/2 #! paper: pole has length 2*s2

# Zeitableitungen der Schwerpunktskoordinaten
Sd1, Sd2  = st.col_split(st.time_deriv(st.col_stack(S1, S2), ttheta)) ##

In [21]:
Sd2

Matrix([
[pdot1*s2*cos(p1)/2 + qdot1],
[       -pdot1*s2*sin(p1)/2]])

In [100]:
k = sp.var("k") # k = 4/3
l = sp.var("l")

In [31]:
# Energie
# T_rot = 0 # (Punktmassenmodell)
# J = k*m2*s2**2
J = sp.var("J")
# T_rot = 0.5 * (J * (Sd2[0]**2 + Sd2[1]**2) / s2**2) # (Verteilte-Masse)
T_rot = 0.5 * (J * tthetad[0]**2) # (Verteilte-Masse)
T_trans = ( m1*Sd1.T*Sd1  +  m2*Sd2.T*Sd2 )/2

T = T_rot + T_trans[0]

V = m2*g*S2[1]

In [56]:
tthetad

Matrix([
[pdot1],
[qdot1]])

In [29]:
sp.simplify(T_rot)

0.5*J*pdot1

In [14]:
T_trans

Matrix([[m1*qdot1**2/2 + m2*pdot1**2*s2**2*sin(p1)**2/8 + m2*(pdot1*s2*cos(p1)/2 + qdot1)**2/2]])

In [57]:
L = T - V
sp.simplify(L)

0.5*J*pdot1**2 - 0.5*g*m2*s2*cos(p1) + 0.5*m1*qdot1**2 + 0.125*m2*pdot1**2*s2**2 + 0.5*m2*pdot1*qdot1*s2*cos(p1) + 0.5*m2*qdot1**2

In [60]:
sp.simplify(st.jac(L, ttheta))

Matrix([[m2*s2*(g - pdot1*qdot1)*sin(p1)/2, 0]])

In [80]:
sp.simplify(st.jac(L, tthetad))

Matrix([[J*pdot1 + m2*pdot1*s2**2/4 + m2*qdot1*s2*cos(p1)/2, m1*qdot1 + m2*pdot1*s2*cos(p1)/2 + m2*qdot1]])

In [83]:
sp.simplify(st.time_deriv(st.jac(L, tthetad), [*ttheta, *tthetad]))

Matrix([[J*pddot1 + m2*pddot1*s2**2/4 - m2*pdot1*qdot1*s2*sin(p1)/2 + m2*qddot1*s2*cos(p1)/2, m1*qddot1 + m2*(pddot1*s2*cos(p1) - pdot1**2*s2*sin(p1) + 2*qddot1)/2]])

In [33]:
mod = mt.generate_symbolic_model(T, V, ttheta, [0, tau1])

In [87]:
mod.eqns

Matrix([
[   J*pddot1 - g*m2*s2*sin(p1)/2 + m2*pddot1*s2**2/4 + m2*qddot1*s2*cos(p1)/2],
[m1*qddot1 + m2*(pddot1*s2*cos(p1) - pdot1**2*s2*sin(p1) + 2*qddot1)/2 - tau1]])

In [94]:
ddx = sp.solve(mod.eqns[1], tthetadd[1])[0]
ddx

(-m2*pddot1*s2*cos(p1)/2 + m2*pdot1**2*s2*sin(p1)/2 + tau1)/(m1 + m2)

In [116]:
(sp.simplify(-mod.eqns[0].subs(tthetadd[1], ddx)).expand() * (4*m1 + 4*m2)).cancel().collect(tthetadd[0])

2*g*m1*m2*s2*sin(p1) + 2*g*m2**2*s2*sin(p1) - m2**2*pdot1**2*s2**2*sin(p1)*cos(p1) - 2*m2*s2*tau1*cos(p1) + pddot1*(-4*J*m1 - 4*J*m2 - m1*m2*s2**2 + m2**2*s2**2*cos(p1)**2 - m2**2*s2**2)

In [117]:
sp.simplify(sp.solve(mod.eqns[0].subs(tthetadd[1], ddx), tthetadd[0])[0].subs(J, k*s2**2*m2).subs(s2, 2*l))

(g*m1*sin(p1) + g*m2*sin(p1) - l*m2*pdot1**2*sin(2*p1)/2 - tau1*cos(p1))/(l*(4*k*m1 + 4*k*m2 + m1 + m2*sin(p1)**2))

In [34]:
mod.MM.simplify()
mod.MM

Matrix([
[ J + m2*s2**2/4, m2*s2*cos(p1)/2],
[m2*s2*cos(p1)/2,         m1 + m2]])

In [35]:
mod.calc_coll_part_lin_state_eq(simplify=True)

In [36]:
mod.calc_state_eq(simplify=True)
sp.cancel(sp.together(mod.state_eq))


Matrix([
[                                                                                                                                                                                  pdot1],
[                                                                                                                                                                                  qdot1],
[                   (2*g*m1*m2*s2*sin(p1) + 2*g*m2**2*s2*sin(p1) - m2**2*pdot1**2*s2**2*sin(p1)*cos(p1) - 2*m2*s2*tau1*cos(p1))/(4*J*m1 + 4*J*m2 + m1*m2*s2**2 + m2**2*s2**2*sin(p1)**2)],
[(4*J*m2*pdot1**2*s2*sin(p1) + 8*J*tau1 - 2*g*m2**2*s2**2*sin(p1)*cos(p1) + m2**2*pdot1**2*s2**3*sin(p1) + 2*m2*s2**2*tau1)/(8*J*m1 + 8*J*m2 + 2*m1*m2*s2**2 + 2*m2**2*s2**2*sin(p1)**2)]])

In [11]:
num, den = sp.together(mod.state_eq)[2].subs(sp.sin(p1)**2, 1-sp.cos(p1)**2).as_numer_denom()
num = sp.factor(num)
num

-2.0*(-g*k*m2*sin(p1) - g*m1*sin(p1) - g*m2*sin(p1) + 2*k**2*m2*pdot1**2*s2*sin(p1)*cos(p1) + 4*k*m2*pdot1**2*s2*sin(p1)*cos(p1) - k*tau1*cos(p1) + 2*m2*pdot1**2*s2*sin(p1)*cos(p1) - tau1*cos(p1))

In [12]:
den = sp.factor(sp.collect(den.expand(), cos(p1)**2))
den

-4.0*s2*(1.0*k + 1.0)*(1.0*k*m2*cos(p1)**2 - 1.0*k*m2 - 1.0*m1 + 1.0*m2*cos(p1)**2 - 1.0*m2)

In [13]:
num / den

0.5*(-g*k*m2*sin(p1) - g*m1*sin(p1) - g*m2*sin(p1) + 2*k**2*m2*pdot1**2*s2*sin(p1)*cos(p1) + 4*k*m2*pdot1**2*s2*sin(p1)*cos(p1) - k*tau1*cos(p1) + 2*m2*pdot1**2*s2*sin(p1)*cos(p1) - tau1*cos(p1))/(s2*(1.0*k + 1.0)*(1.0*k*m2*cos(p1)**2 - 1.0*k*m2 - 1.0*m1 + 1.0*m2*cos(p1)**2 - 1.0*m2))

In [41]:
xx = mod.x 
f = mod.ff 
G = mod.gg 
g1, = st.col_split(G)

In [29]:
f

Matrix([
[                   pdot1],
[                   qdot1],
[2*g*sin(p1)/(s2*(k + 1))],
[                       0]])

In [30]:
G

Matrix([
[            0],
[            0],
[-2*cos(p1)/s2],
[            1]])

In [31]:
eqp1 = sp.Matrix([0, 0, 0, 0 ]) ##:T
eqp2 = sp.Matrix([sp.pi, 0, 0, 0]) ##:T

# Probe:
assert f.subz(xx, eqp1) == sp.zeros(4,1)
assert f.subz(xx, eqp2) == sp.zeros(4,1)

In [36]:
# Parameterwerte
parameter_values = [(g, 9.81), (s2, .5), (k, 4/3)]
replm =  parameter_values + list(zip(xx, eqp1))

In [37]:
A1 = f.jacobian(xx).subs(replm) 
A1

Matrix([
[               0, 0, 1, 0],
[               0, 0, 0, 1],
[16.8171428571429, 0, 0, 0],
[               0, 0, 0, 0]])

In [38]:
b1 = G.subs(replm) 
b1

Matrix([
[   0],
[   0],
[-4.0],
[   1]])

In [39]:
a = 1.2
b = 3.4

# poles = np.r_[-1.5+a*1j, -1.5-a*1j, -1.3 + b*1j, -1.3 - b*1j] ##:
poles = np.linspace(-5, -8, 4)
# k1 = st.siso_place(A1, b1, poles)
# print(k1)
# A1-b1@k1
import control as ctrl
F = ctrl.place(np.array(A1,dtype=float), np.array(b1,dtype=float), poles)
print("F", F)
np.linalg.eigvals(np.array(A1, dtype=float)- np.array(b1,dtype=float)@F)


F [[-91.92880151 -99.8980632  -22.34692491 -63.38769963]]


array([-8., -7., -6., -5.])

In [ ]:
(A1-b1@k1.T).eigenvals()

{34.4651505758603 - 3.31186942765839e-64*I: 1,
 -2.68043452853918 - 2.9185170576*I: 1,
 -3.10428151878197 + 2.55627288704323e-63*I: 1,
 -2.68043452853918 + 2.9185170576*I: 1}

## Equation of Paper
https://coneural.org/florian/papers/05_cart_pole.pdf

In [37]:
phi, phi_dot, phi_dot_dot, x, x_dot, x_dot_dot, k = sp.var("phi, phi_dot, phi_dot_dot, x, x_dot, x_dot_dot, k")
phi_dot_dot = (g*sin(phi)* (m1+m2) + cos(phi) * ((-tau1 - m2*s2*phi_dot**2*sin(phi)))) / (s2*(k*(m1+m2)-(m2*cos(phi)**2)))
x_dot_dot = sp.simplify((tau1 + m2*s2*(phi_dot**2*sin(phi) - phi_dot_dot*cos(phi))) / (m1+m2))
phi_dot_dot

(g*(m1 + m2)*sin(phi) + (-m2*phi_dot**2*s2*sin(phi) - tau1)*cos(phi))/(s2*(k*(m1 + m2) - m2*cos(phi)**2))

In [38]:
x_dot_dot

(-g*m2*sin(2*phi)/2 + k*m2*phi_dot**2*s2*sin(phi) + k*tau1)/(k*m1 + k*m2 - m2*cos(phi)**2)

In [39]:
xx2 = sp.Matrix([[phi, x, phi_dot, x_dot]]).T
xx2

Matrix([
[    phi],
[      x],
[phi_dot],
[  x_dot]])

In [42]:
sys = sp.Matrix([[phi_dot, x_dot, phi_dot_dot, x_dot_dot]]).T.subs(zip(xx2, xx))
sys

Matrix([
[                                                                                              pdot1],
[                                                                                              qdot1],
[(g*(m1 + m2)*sin(p1) + (-m2*pdot1**2*s2*sin(p1) - tau1)*cos(p1))/(s2*(k*(m1 + m2) - m2*cos(p1)**2))],
[              (-g*m2*sin(2*p1)/2 + k*m2*pdot1**2*s2*sin(p1) + k*tau1)/(k*m1 + k*m2 - m2*cos(p1)**2)]])

In [46]:
num2, den2 = sys[2].as_numer_denom()
eq1 = num2 + tthetadd[0] * den2
num3, den3 = sys[3].as_numer_denom()
eq2 = num3 + tthetadd[1] * den3
eq2

-g*m2*sin(2*p1) + 2*k*m2*pdot1**2*s2*sin(p1) + 2*k*tau1 + qddot1*(2*k*m1 + 2*k*m2 - 2*m2*cos(p1)**2)

In [53]:
mod.eqns.subs(J, k*m2*s2**2).expand()

Matrix([
[-g*m2*s2*sin(p1)/2 + k*m2*pddot1*s2**2 + m2*pddot1*s2**2/4 + m2*qddot1*s2*cos(p1)/2],
[   m1*qddot1 + m2*pddot1*s2*cos(p1)/2 - m2*pdot1**2*s2*sin(p1)/2 + m2*qddot1 - tau1]])

In [50]:
mod2.eqns

Matrix([
[g*(m1 + m2)*sin(p1) + pddot1*s2*(k*(m1 + m2) - m2*cos(p1)**2) + (-m2*pdot1**2*s2*sin(p1) - tau1)*cos(p1)],
[    -g*m2*sin(2*p1) + 2*k*m2*pdot1**2*s2*sin(p1) + 2*k*tau1 + qddot1*(2*k*m1 + 2*k*m2 - 2*m2*cos(p1)**2)]])

In [44]:
mod2 = mt.SymbolicModel()

In [ ]:
type(mod.eqns)

sympy.matrices.dense.MutableDenseMatrix

In [47]:
mod2.tt = ttheta
mod2.ttd = tthetad
mod2.ttdd = tthetadd
mod2.tau = [tau1]
mod2.eqns = sp.Matrix([eq1, eq2])
mod2.x = xx
# mod2.state_eq = sys
mod2.calc_coll_part_lin_state_eq(simplify=True)
mod2.calc_state_eq()

In [48]:
mod2.state_eq

Matrix([
[                                                                                                                                        pdot1],
[                                                                                                                                        qdot1],
[(-g*(m1 + m2)*sin(p1) - (-m2*pdot1**2*s2*sin(p1) - tau1)*cos(p1))*(2*k*m1 + 2*k*m2 - 2*m2*cos(p1)**2)/(2*s2*(k*m1 + k*m2 - m2*cos(p1)**2)**2)],
[                  (k*(m1 + m2) - m2*cos(p1)**2)*(g*m2*sin(2*p1) - 2*k*m2*pdot1**2*s2*sin(p1) - 2*k*tau1)/(2*(k*m1 + k*m2 - m2*cos(p1)**2)**2)]])

In [ ]:
repl = parameter_values + list(zip(xx2, eqp1)) + [(m1, 1), (m2, 0.1), (k, 4/3)]

In [ ]:
A2 = sys.jacobian(xx2).subs(repl) 
A2


Matrix([
[                 0, 0, 1, 0],
[                 0, 0, 0, 1],
[  15.7917073170732, 0, 0, 0],
[-0.717804878048781, 0, 0, 0]])

In [ ]:
B2 = - sys.jacobian([tau1]).subs(repl) 
B2

Matrix([
[                 0],
[                 0],
[  1.46341463414634],
[-0.975609756097561]])

In [ ]:
F2 = ctrl.place(np.array(A2,dtype=float), np.array(B2, dtype=float), np.linspace(-6, -8, 4))
F2

array([[319.9059545 , 163.46132065,  82.12373114,  94.48559671]])

In [ ]:
(A2-B2@F2).eigenvals()

{-8.00000000003526 - 3.18212098035427e-65*I: 1,
 -5.99999999996732 - 1.14390484568382e-64*I: 1,
 -7.33333333314472 + 2.61435543279885e-64*I: 1,
 -6.66666666675365 + 3.8311135891318e-67*I: 1}

Vergleich oben unten

In [ ]:
diff = sp.simplify(mod2.state_eq.subs(list(zip(xx2, xx))) - mod.state_eq)

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                       0],
[                                                                                                                                                                                                                                                                                                                                                                                                                       0],
[0.25*(2.0*(k*(m1 + m2) - m2*cos(p1)**2)*(-g*(k*m2 + m1 + m2)*sin(p1) + (k + 1)*(2*k*m2*pdot1**2*s2*sin(p1) + 2*m2*pdot1**2*s2*sin(p1) - tau1)*cos(p1)) + (g*(m1 + m2)*

In [ ]:
mod2.state_eq.subs(list(zip(xx2, xx)))

Matrix([
[                                                                                              pdot1],
[                                                                                              qdot1],
[(g*(m1 + m2)*sin(p1) + (-m2*pdot1**2*s2*sin(p1) - tau1)*cos(p1))/(s2*(k*(m1 + m2) - m2*cos(p1)**2))],
[              (-g*m2*sin(2*p1)/2 + k*m2*pdot1**2*s2*sin(p1) + k*tau1)/(k*m1 + k*m2 - m2*cos(p1)**2)]])